# Computer Vision Project : Enhancing Nucleus Segmentation and 3D Reconstruction Using Super-Resolution


## Group Members:
### Rita Sulaiman – Student ID: 2210765051
### Zeynep Yıldız – Student ID: 2210765033
### Zharasbek Bimagambetov – Student ID: 2210356185


## **Load & Preprocess the Dataset**

### Imports

In [4]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

import cv2
import numpy as np
import matplotlib.pyplot as plt

import torch
import os
from PIL import Image
from torchvision import transforms
import matplotlib.pyplot as plt
import numpy as np
import cv2
from tqdm import tqdm
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torchvision.transforms import functional as TF
from PIL import Image
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np
from skimage.metrics import peak_signal_noise_ratio, structural_similarity
import torch.nn.functional as F
import cv2
from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import structural_similarity as ssim


### Preprocessing and Image Display

In [3]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from sklearn.model_selection import train_test_split
from PIL import Image

# ================================
# CONFIGURATION
# ================================

RAW_ROOT = Path(r"C:\Users\rita\.cache\kagglehub\datasets\ipateam\nuinsseg\versions\5")
OUT_ROOT = Path("data")
SPLITS = ["train", "val", "test"]
TARGET_SIZE = (256, 256)
test_frac = 0.10
val_frac = 0.10
random_seed = 42

# Create output directory structure
for split in SPLITS:
    for sub in ["images", "masks", "distance_maps", "label_masks", "vague_masks"]:
        (OUT_ROOT / split / sub).mkdir(parents=True, exist_ok=True)

print("✅ Directory scaffold created under", OUT_ROOT)

# ================================
# COLLECT VALID TUPLES
# ================================

data_tuples = []
for organ in RAW_ROOT.iterdir():
    if not organ.is_dir():
        continue

    tissue_dir = organ / "tissue images"
    mask_dir   = organ / "mask binary"
    dist_dir   = organ / "distance maps"
    label_dir  = organ / "label masks modify"
    vague_dir  = organ / "vague areas" / "mask binary"

    for d in [tissue_dir, mask_dir, dist_dir, label_dir, vague_dir]:
        if not d.exists():
            print(f"⚠️ Skipping {organ.name}: missing {d}")
            break
    else:
        for img_path in tissue_dir.glob("*.png"):
            stem = img_path.stem
            m1 = mask_dir / f"{stem}.png"
            m2 = dist_dir / f"{stem}.png"
            m3 = label_dir / f"{stem}.tif"
            m4 = vague_dir / f"{stem}.png"
            if m1.exists() and m2.exists() and m3.exists() and m4.exists():
                data_tuples.append((img_path, m1, m2, m3, m4))

print(f"✅ Found {len(data_tuples)} complete data tuples.")

# ================================
# SPLIT DATA
# ================================

train_val, test = train_test_split(data_tuples, test_size=test_frac, random_state=random_seed)
train, val = train_test_split(train_val, test_size=val_frac / (1 - test_frac), random_state=random_seed)
print(f"📊 Split sizes — Train: {len(train)}, Val: {len(val)}, Test: {len(test)}")

# ================================
# PROCESS & SAVE RESIZED OUTPUT
# ================================

def process_and_save(split_list, split_name):
    for (img, msk, dist, lbl, vmask) in split_list:
        # Load all
        arr_img  = cv2.imread(str(img))
        arr_msk  = cv2.imread(str(msk),  cv2.IMREAD_GRAYSCALE)
        arr_dist = cv2.imread(str(dist), cv2.IMREAD_GRAYSCALE)
        arr_lbl  = cv2.imread(str(lbl),  cv2.IMREAD_UNCHANGED)
        arr_v    = cv2.imread(str(vmask), cv2.IMREAD_GRAYSCALE)

        # Resize all
        img_r  = cv2.resize(arr_img,  TARGET_SIZE, interpolation=cv2.INTER_CUBIC)
        msk_r  = cv2.resize(arr_msk,  TARGET_SIZE, interpolation=cv2.INTER_NEAREST)
        dist_r = cv2.resize(arr_dist, TARGET_SIZE, interpolation=cv2.INTER_NEAREST)
        lbl_r  = cv2.resize(arr_lbl,  TARGET_SIZE, interpolation=cv2.INTER_NEAREST)
        v_r    = cv2.resize(arr_v,    TARGET_SIZE, interpolation=cv2.INTER_NEAREST)

        # Save
        cv2.imwrite(str(OUT_ROOT/split_name/"images"/ img.name),  img_r)
        cv2.imwrite(str(OUT_ROOT/split_name/"masks"/  msk.name),  msk_r)
        cv2.imwrite(str(OUT_ROOT/split_name/"distance_maps"/ dist.name), dist_r)
        cv2.imwrite(str(OUT_ROOT/split_name/"label_masks"/ lbl.name), lbl_r)
        cv2.imwrite(str(OUT_ROOT/split_name/"vague_masks"/ vmask.name),  v_r)

    print(f"✅ {split_name} saved: {len(split_list)} samples")

process_and_save(train, "train")
process_and_save(val,   "val")
process_and_save(test,  "test")

# Confirm counts
for split in SPLITS:
    counts = {sub: len(list((OUT_ROOT/split/sub).glob("*.*"))) for sub in ["images", "masks", "distance_maps", "label_masks", "vague_masks"]}
    print(f"{split} counts: ", counts)


✅ Directory scaffold created under data
✅ Found 665 complete data tuples.
📊 Split sizes — Train: 531, Val: 67, Test: 67
✅ train saved: 531 samples
✅ val saved: 67 samples
✅ test saved: 67 samples
train counts:  {'images': 531, 'masks': 531, 'distance_maps': 531, 'label_masks': 531, 'vague_masks': 531}
val counts:  {'images': 67, 'masks': 67, 'distance_maps': 67, 'label_masks': 67, 'vague_masks': 67}
test counts:  {'images': 67, 'masks': 67, 'distance_maps': 67, 'label_masks': 67, 'vague_masks': 67}


### Downsampling

In [ ]:
##done in safe_downsample.py
## Run using terminal "python safe_downsample.py"

## **Super Resolution**

In [ ]:
# ========== CONFIG ==========
downsampled_folder = "downsampled_data/train/images"
original_folder = "data/train/images"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Paths to models
model_paths = {
    "srcnn": "models/srcnn_model.pth",
    "espcn": "models/espcn_x2_256to512.pth",
    "edsr_me": "models/edsr_x2_256to512.pth",
    "edsr": "models/EDSR_x2.pb"
}

# Output dictionary to store results
results = {}


In [5]:
# Define SRCNN
class SRCNN(nn.Module):
    def __init__(self):
        super(SRCNN, self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=9, padding=4),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 32, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.Conv2d(32, 3, kernel_size=5, padding=2)
        )

    def forward(self, x):
        return self.model(x)


# Define SRDFN
class ResidualBlock(nn.Module):
    def __init__(self, num_channels):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Conv2d(num_channels, num_channels, kernel_size=3, padding=1)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(num_channels, num_channels, kernel_size=3, padding=1)

    def forward(self, x):
        residual = x
        out = self.relu(self.conv1(x))
        out = self.conv2(out)
        return out + residual  # Skip connection

class EDSR(nn.Module):
    def __init__(self, scale_factor=2, num_channels=64, num_blocks=8):
        super(EDSR, self).__init__()
        self.entry = nn.Conv2d(3, num_channels, kernel_size=3, padding=1)

        self.res_blocks = nn.Sequential(
            *[ResidualBlock(num_channels) for _ in range(num_blocks)]
        )

        self.conv = nn.Conv2d(num_channels, num_channels, kernel_size=3, padding=1)
        self.upsample = nn.Sequential(
            nn.Conv2d(num_channels, 3 * (scale_factor ** 2), kernel_size=3, padding=1),
            nn.PixelShuffle(scale_factor)
        )

    def forward(self, x):
        x = self.entry(x)
        res = self.res_blocks(x)
        x = self.conv(res) + x  # Global skip connection
        x = self.upsample(x)
        return x
    

# Define ESPCN
class ESPCN(nn.Module):
    def __init__(self, scale_factor=3):
        super(ESPCN, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=5, padding=2)
        self.conv2 = nn.Conv2d(64, 32, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(32, 3 * (scale_factor ** 2), kernel_size=3, padding=1)
        self.pixel_shuffle = nn.PixelShuffle(scale_factor)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = self.conv3(x)
        x = self.pixel_shuffle(x)
        return x



In [6]:
def apply_sr(model_name, img_path):
    image = Image.open(img_path).convert("RGB")
    input_tensor = transforms.ToTensor()(image).unsqueeze(0).to(device)

    # Directly load whole model object
    model = torch.load(model_paths[model_name], map_location=device)
    model.eval()

    with torch.no_grad():
        sr = model(input_tensor)

    sr_img = sr.squeeze(0).permute(1, 2, 0).cpu().numpy()
    return np.clip(sr_img, 0, 1)



In [7]:
model_names = ["srcnn", "espcn", "edsr_me"]

for model_name in model_names:
    psnrs = []
    ssims = []

    print(f"🔍 Testing {model_name.upper()}...")

    for filename in tqdm(os.listdir(downsampled_folder)):
        lr_path = os.path.join(downsampled_folder, filename)
        hr_path = os.path.join(original_folder, filename)

        if not os.path.exists(hr_path):
            continue

        # Load images
        lr_img = Image.open(lr_path).convert("RGB")
        hr_img = Image.open(hr_path).convert("RGB")

        # Apply super-resolution
        
        sr_img = apply_sr(model_name, lr_path)

        # Resize to match HR image
        sr_img_resized = np.array(Image.fromarray((sr_img * 255).astype(np.uint8)).resize(hr_img.size)).astype(np.float32) / 255.0
        hr_np = np.array(hr_img).astype(np.float32) / 255.0

        # Metrics
        psnr = peak_signal_noise_ratio(hr_np, sr_img_resized, data_range=1.0)
        ssim = structural_similarity(hr_np, sr_img_resized, channel_axis=-1, data_range=1.0)
        psnrs.append(psnr)
        ssims.append(ssim)

    results[model_name] = {
        "PSNR": np.mean(psnrs),
        "SSIM": np.mean(ssims)
    }

    print(f"✅ {model_name.upper()} — PSNR: {np.mean(psnrs):.2f} | SSIM: {np.mean(ssims):.4f}")


🔍 Testing SRCNN...


  0%|          | 0/531 [00:00<?, ?it/s]C:\Users\rita\AppData\Local\Temp\ipykernel_17528\1356720203.py:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load(mod

FileNotFoundError: [Errno 2] No such file or directory: 'models/srcnn_model.pth'

In [53]:
print("\n📊 Final Comparison:")
for name, metrics in results.items():
    print(f"{name.upper():<10} | PSNR: {metrics['PSNR']:.2f} | SSIM: {metrics['SSIM']:.4f}")



📊 Final Comparison:
SRCNN      | PSNR: 33.04 | SSIM: 0.7932
ESPCN      | PSNR: 35.74 | SSIM: 0.8519
EDSR_ME    | PSNR: 36.25 | SSIM: 0.8673
